## This is the MVP for the initial project. From here it was paired down to only EA data to focus
## on a predictable reliable data set and save time as there were several issues scraping.

*ATVI, TTWO, EA
*Ubisoft (UBSFF, UBSFY)
*THQ (THQI - bankrupt, THQQF, THQN B)
*Japanese? Konami (KNM / KNMCY), Nintendo (NTDOY, 7974 JP), Capcom (CCOEF, 9697 JP)
*Smartphones? KING, ZNGA, GLUU

In [7]:
from __future__ import print_function, division
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from bs4 import BeautifulSoup
import re
import pandas as pd
import random

In [8]:
#initiate driver window
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
url = 'https://www.gameinformer.com/reviews'
driver.get(url)

In [9]:
page_counter = 1
count_down = 150 #120 pages as of 7/18/2018

# loop this a bunch of times w a time element to pause
while count_down > 0:
    try:
        next_page_button=driver.find_element_by_xpath('//*[@id="block-gi5-content"]/div/div/ul/li/a')
        next_page_button.click()
        page_counter = page_counter+1
    except:
        print("Loaded",page_counter,"pages")
    
    count_down = count_down -1
    print(count_down)    
    
    time.sleep(random.randint(6,10)) #wired at 6-10s; wi-fi was 25-40 at Metis
   

249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
Loaded 120 pages
130
Loaded 120 pages
129
Loaded 120 pages
128
Loaded 120 pages
127
Loaded 120 pages
126
Loaded 120 pages
125
Loaded 120 pages
124
Loaded 120 pages
123
Loaded 120 pages
122
Loaded 120 pages
121
Loaded 120 pages
120
Loaded 120 pages
119
Loaded 120 pages
118
Loaded 120 pages
117
Loaded 120 pages
116
Loaded 120 pages
115
Loaded 120 pages
114
Loaded 120 pages
113
Loaded 120 pages
112
Loaded 120 pages
111
Loaded 120 pages
110
Loaded 120 pages
109
Loaded 120 pages
108
Loaded 120 pages
107
Loaded 120 pages
106

In [183]:
soup = BeautifulSoup(driver.page_source, "lxml")

# But we are missing some of the scores....}

In [184]:
elements = soup.find_all("div", class_="teaser-left".split()) #grab all the scores & null

In [185]:
#loop through elements, for errors substitute 5.0 (neutral score)
scores=[]
for el in elements:
    try: 
        scores.append(float(el.find("div",{"class":"score"}).get_text().strip()))
    except:
        scores.append(5.0)

In [186]:
#scrape the titles and dates

titles = soup.find_all("h2",{"class":"page-title"})
titles = [title.get_text().strip() for title in titles]

author_dates = soup.find_all("div",{"class":"author-details"})
author_dates = [author.get_text().replace('\t','').replace('\n',' ').replace('\xa0','') for author in author_dates]

#clean strings
start = ''
end = ' Review'
titles  = [re.search('%s(.*)%s' % (start, end),title) for title in titles]

start = 'by '
end = ' on'
authors  = [re.search('%s(.*)%s' % (start, end),author).group(1) for author in author_dates]

start = 'on '
end = ' at'
dates  = [re.search('%s(.*)%s' % (start, end),date).group(1) for date in author_dates]

In [187]:
#trim dates to 12 characters on right
dates_clean = []
for item in dates:
    if len(item)==12:
        dates_clean.append(item)
    else:
        dates_clean.append(item[-12:])

In [188]:
#handle missing titles
newlist=[]
start = ''
end = ' Review'
for title in titles:
    try:
        newlist.append(title.group(1))
    except:
        newlist.append('Missing')

titles = newlist
print(dates)

[]


In [128]:
#building the dataframe from scraped data
#reviews = pd.DataFrame(
{
    'Date': dates_clean,'Title': titles,'Score': scores
})

clean_shit=[]
for item in mattframe['fuckedup']:
    item_string = str(item)
    item_string.replace('fuckedup','').replace('+ unchd','')
    item_string[-11:]   #last 11 characters of string
    item_string.append(clean_shit)

clean_shit


pd.DataFrame({
    'Clean_shit': clean_shit,'Clean_shit2': clean_shit2,'Score': scores
    }) 

In [129]:
#saved 120 pages of scraped data to csv 7/18/18
#reviews.to_csv('reviews.csv',index=False)

In [196]:
#if you need to reload data from CSV:
reviews = pd.read_csv('reviews.csv')
reviews.head()

,Date,Score,Title
0,"Jul 17, 2018",6.0,Adventure Time: Pirates Of The Enchiridion
1,"Jul 17, 2018",6.0,Adventure Time: Pirates Of The Enchiridion
2,"Jul 17, 2018",6.0,Adventure Time: Pirates Of The Enchiridion
3,"Jul 13, 2018",5.0,Earthfall
4,"Jul 13, 2018",8.0,Captain Toad: Treasure Tracker


In [218]:
class HTMLTableParser:
   
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  
    
    def parse_soup(self, soup):
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [219]:
# Scrape legacy table data
url = 'https://gi9641r1.cachefly.net/reviewsarchive/review-archive-legacy-pantheon14.html'
driver.get(url)
soup_level1=BeautifulSoup(driver.page_source, 'lxml')
hp = HTMLTableParser()
legacy_table = hp.parse_soup(soup_level1)[0][1] # Grabbing the table from the tuple

ATVI, TTWO, EA, THQI, Ubisoft game data:
1. https://en.wikipedia.org/wiki/List_of_Activision_video_games
2. https://en.wikipedia.org/wiki/List_of_Take-Two_Interactive_games
3. https://en.wikipedia.org/wiki/List_of_Electronic_Arts_games
4. https://en.wikipedia.org/wiki/List_of_THQ_games
5. https://en.wikipedia.org/wiki/List_of_Ubisoft_games

Download stock returns from wsj.com:
https://quotes.wsj.com/company-list

In [194]:
#import game list from wikipedia scrapes
games = pd.read_csv('games.csv')

In [220]:
#drop excess columns
less_legacy['Date']=legacy_table['Issue']
less_legacy = less_legacy.drop(columns=['Platform','Issue'])
less_games = games.drop(columns=['Year','Platform','Date'])

ValueError: labels ['Platform' 'Issue'] not contained in axis

In [224]:
# stack the DataFrames on top of each other so we can drop dupes from top to bottom
all_reviews = pd.concat([reviews, less_legacy], axis=0)

In [231]:
all_reviews.head()

,Date,Score,Title
0,"Jul 17, 2018",6,Adventure Time: Pirates Of The Enchiridion
1,"Jul 17, 2018",6,Adventure Time: Pirates Of The Enchiridion
2,"Jul 17, 2018",6,Adventure Time: Pirates Of The Enchiridion
3,"Jul 13, 2018",5,Earthfall
4,"Jul 13, 2018",8,Captain Toad: Treasure Tracker


In [276]:
#Get rid of dupes from the two gameinformer's datasets
clean_reviews = all_reviews.drop_duplicates(subset='Title',keep='first')

#Get rid of dupes from multiple-platform entries in less_games
clean_games = less_games.drop_duplicates(subset='Title',keep='first')

In [277]:
#dropped 3k dupe-reviews; 4k dupe-games
clean_reviews.shape

(7551, 3)

In [269]:
clean_reviews.Title.value_counts()  #result looks reasonable from a single source (gameinformer)
clean_games.Title.value_counts() #only public company's games

Simcoaster                                                1
Castlevania: Legacy of Darkness                           1
Rollcage                                                  1
DK: Jungle Climer                                         1
Tales From The Borderlands: Episode 1 – Zero Sum          1
Hot Shots Golf 2                                          1
Sonic  Colors Wii                                         1
Aggressive Inline                                         1
Championship Motocross 2001 Featuring Ricky Carmichael    1
Folklore                                                  1
Devil May Cry 3: Dante's Awakening                        1
Plants Vs. Zombies: Garden Warfare 2                      1
Panic                                                     1
Weapon Lord                                               1
NBA Live 09                                               1
Orcs Must Die!                                            1
Robopon 2: Ring Version                 

In [281]:
#clean_reviews = consolidated gameinformer reviews
#clean_games = games by publically traded co's
#inner join to keep only games w/ reviews by specific publishers
merged_reviews = pd.merge(clean_reviews, clean_games, how = 'inner',on='Title')  

In [305]:
merged_reviews['Date'] = pd.to_datetime(merged_reviews['Date'])

In [442]:
#compile relative returns into a single data frame
ATVI = pd.read_csv('ATVI.csv')
EA = pd.read_csv('EA.csv')
TTWO = pd.read_csv('TTWO.csv')
UBI = pd.read_csv('UBI.csv')
THQI = pd.read_csv('THQI.csv')
all_stocks = pd.concat([ATVI, EA, TTWO, UBI, THQI], axis=0)

In [443]:
mini_stocks = all_stocks.drop(columns=['Date+7','Date+30',' Close','Ticker','SPX+30','SPX+7','Stock+7','Stock+30'])
mini_stocks['Date']= pd.to_datetime(mini_stocks['Date'])

In [563]:
mini_stocks.describe()

,rel_return_7d,rel_return_30d
count,17801.000000,17801.000000
mean,0.307067,1.559146
std,3.531911,5.827906
min,-74.830000,-73.510000
25%,0.307739,1.413024
50%,0.410000,2.010000
75%,0.502833,2.525639
max,62.840000,49.180000


In [565]:
#Replacing missing return values using long term average return for each publisher
mini_stocks["rel_return_7d"] = mini_stocks.groupby("Publisher")['rel_return_7d'].transform(lambda x: x.fillna(0.307067))
mini_stocks["rel_return_30d"] = mini_stocks.groupby("Publisher")['rel_return_30d'].transform(lambda x: x.fillna(5.8279))

In [566]:
merged_reviews.describe()

,Date,Score,Title,Publisher
count,7551,7551,7551,1055
unique,1441,74,7551,5
top,2003-01,8,Simcoaster,EA
freq,111,605,1,431


In [437]:
data = pd.merge(merged_reviews, mini_stocks, on=['Date','Publisher'], how ="left")  

In [440]:
data["rel_return_30d"] = data.groupby("Publisher")['rel_return_30d'].transform(lambda x: x.fillna(x.mean()))
data["rel_return_7d"] = data.groupby("Publisher")['rel_return_7d'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
data